In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# PETScDirectSolver

PETScDirectSolver is a linear solver that assembles the system Jacobian and solves the linear system with LU factorization and back substitution using the `petsc4py` library. It can handle any system topology. Since it assembles a global Jacobian for all of its subsystems, any linear solver that is assigned in any of its subsystems does not participate in this calculation (though they may be used in other ways such as in subsystem Newton solves). Unlike the standard DirectSolver which always uses SuperLU (for sparse) or LAPACK (dense) through scipy, the PETScDirectSolver has access to multiple DirectSolvers available in PETSc.

PETSc is more general and has more options than scipy for direct solvers, but due to the generality PETSc provides a thicker wrapper around its solvers. So when considering whether to use `DirectSolver` or `PETScDirectSolver`, one should consider the size of their problem and sparsity pattern. Typically PETSc will be more beneficial for larger matrices where the factorization / solving speedup from a different solver is larger than the slowdown due to overhead from the heavier wrapper. For more info about the solvers exposed by the PETScDirectSolver, see this summary table of direct solvers from the [PETSc documentation](https://petsc.org/release/overview/linear_solve_table/#direct-solvers).

Of the available sparse direct solvers, SuperLU, KLU, UMFPACK, and PETSc can only be used in serial. The MUMPS and SuperLU-Dist solvers can be used in serial, but will also be run distributed if the script is run with MPI. For dense matrices the solver will ignore the sparse algorithms and automatically use LAPACK, which only runs in serial.

As an example, here we calculate the total derivatives of the Sellar system objective with respect to the design variable 'z', using the "klu" direct solver in the PETScDirectSolver.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src23", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src23`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
model = prob.model = SellarDerivatives()

model.nonlinear_solver = om.NonlinearBlockGS()
model.linear_solver = om.PETScDirectSolver(sparse_solver_name='klu')

prob.setup()
prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(J['obj', 'z'][0][0], 9.61001056, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78448534, .00001)

## PETScDirectSolver Options

In [ ]:
om.show_options_table("openmdao.solvers.linear.petsc_direct_solver.PETScDirectSolver")

## PETScDirectSolver Constructor

The call signature for the `PETScDirectSolver` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.linear.petsc_direct_solver.PETScDirectSolver.__init__
        :noindex:
```